In [91]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [92]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
for (i, job) in enumerate(jobs_list):
    c = 0
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(employment_date), end_date, 1)[0]
            c += 1
        else:
            dismissal_date = None

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, birth_date, pesel,
                                                         city, address, phone, email, employment_date,
                                                         dismissal_date, job, wage]

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [93]:
employees_df

,id,imię,nazwisko,data_urodzenia,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Anna,Moroz,27/09/1971,71092792562,Wrocław,ul. Sportowa 16/36,48062789196,anna.moroz@kuba.pl,2014-01-01,None,menadżer,62.2
1,2,Robert,Łucek,12/06/1992,92061228510,Wałbrzych,ul. Błotnia 92,48799162548,robert.lucek@kuba.pl,2014-01-01,2016-02-26,sprzedawca,54.7
2,3,Henri,Gołuch,14/05/1964,64051483674,Wrocław,ul. Strumykowa 10/18,48808989022,henri.goluch@kuba.pl,2014-01-01,None,sprzedawca,52.8
4,4,Serhii,Dzwonek,29/09/1973,73092999031,Wrocław,ul. Zielona 28/46,48140033355,serhii.dzwonek@kuba.pl,2014-01-01,None,mechanik,52.1
13,5,Antoni,Olejniczak,25/04/1987,87042531237,Wrocław,ul. Nektarowa 80,48287416257,antoni.olejniczak@kuba.pl,2014-01-01,None,informatyk,49.8
3,6,Agnieszka,Siadul,25/10/1988,88102549968,Wrocław,ul. Oliwkowa 52/9,48692199017,agnieszka.siadul@kuba.pl,2014-10-23,None,sprzedawca,52.0
5,7,Katarzyna,Zambok,19/04/1981,81041906209,Pasłęk,ul. gen. Tadeusza Piskora 65/2,48603120635,katarzyna.zambok@kuba.pl,2015-03-20,None,mechanik,59.9
6,8,Olena,Krasoń,17/08/1977,77081782844,Łęczyca,ul. Onufrego Zagłoby 95/16,48213499474,olena.krason@kuba.pl,2015-11-08,None,mechanik,58.1
7,9,Aleksander,Samiczak,29/03/1974,74032933274,Wrocław,ul. Henryka Sienkiewicza 19,48913364151,aleksander.samiczak@kuba.pl,2016-01-22,2017-05-18,mechanik,46.6
8,10,Daniel,Lis,05/04/1967,67040553673,Włocławek,ul. Zabełkowska 82/4,48576154910,daniel.lis@kuba.pl,2016-05-12,None,pomocnik mechanika,44.7


# Transakcje

In [94]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [95]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [96]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [97]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [98]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [99]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [100]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [101]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [102]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        pesel, birth_date = random_pesel_date_man(year_lims)
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        pesel, birth_date = random_pesel_date_woman(year_lims)
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]

In [103]:
clients_df

,id,imię,nazwisko,data_urodzenia,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Martyna,Buśko,31/08/2005,F,05283171968,Wrocław,ul. Dobrodzieńska 22/10,False,48570241578,busko6@wp.pl
1,2,Omina,Gromala,11/05/1989,F,89051110269,Wrocław,ul. Koronowska 10/36,False,48494360013,tt1s9lp91@gmail.com
2,3,Wioletta,Matysek,25/01/1996,F,96012579762,Wrocław,ul. Szewska 52/3,False,48860660288,w-matysek457@gmail.com
3,4,Janina,Brzeżawska,11/06/1989,F,89061111049,Nysa,ul. Kolejowa 7/12,True,48817441044,brzezawska4@vp.pl
4,5,Zenon,Mazurek,29/10/1983,M,83102977456,Wrocław,ul. Karola Szymanowskiego 19,False,48609127140,mazurek0@interia.pl
...,...,...,...,...,...,...,...,...,...,...,...
1507,1508,Janusz,Mazurkiewicz,03/11/1975,M,75110337214,Olsztyn,ul. Chyża 33,False,48225196605,j_mazurkiew75@gmail.com
1508,1509,Gabriela,Siwińska,12/11/1982,F,82111202784,Radoszyce,ul. Jana Czecha 16,False,48875678599,gsiwinska1982@o2.pl
1509,1510,Natalia,Korcz,16/11/1998,F,98111629861,Wrocław,ul. Opłotki 53,False,48797707549,natkorc1998@onet.pl
1510,1511,Oliwia,Dwojewska,24/02/1993,F,93022473066,Chełmża,Rynek 12/20,False,48891584402,dwojewska36594@onet.pl


In [104]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(78)

# auta

In [105]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [106]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [107]:
df_samochody = create_car_table(max(car_ids))

In [108]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Ford,Focus,4,2008,SUV,False,czerwony,Benzyna,1.6,101,Manualna,5,5,False
1,2,Skoda,Rapid/Spaceback,4,2009,Coupe,False,czarny,Benzyna,1.9,122,Manualna,3,2,False
2,3,Volkswagen,T-Cross,4,2008,Sedan,False,czarny,Benzyna,1.5,95,Automatyczna,5,5,False
3,4,Volkswagen,Golf,4,2006,Sedan,False,niebieski,Benzyna,2.8,170,Automatyczna,5,5,False
4,5,Renault,Captur,4,1999,SUV,True,brązowy,Diesel,2.0,140,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,982,BMW,116,4,2007,Kombi,False,szary,Benzyna,1.8,116,Manualna,3,5,False
982,983,Toyota,Aygo,4,2002,Sedan,False,biały,Benzyna,1.0,72,Manualna,5,5,True
983,984,Bajaj,Avenger Cruise 220,2,2010,,False,czarny,Benzyna,0.6,84,Automatyczna,0,1,True
984,985,Volkswagen,up!,4,2010,Sedan,False,czarny,Benzyna,0.9,68,Manualna,5,5,True


# Usługi

In [109]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksiegowania'] = transaction_dates_service

# Zakup samochodów

In [120]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_6932\1503429122.py:2: RuntimeWarning: invalid value encountered in sqrt
  return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -1)
C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_6932\1503429122.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -1)


In [121]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksiegowania'] = transaction_dates_buy

In [ ]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksiegowania'] = service_date_fix['data_zaksiegowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksiegowania'] = buy_date_fix['data_zaksiegowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) > str(buy_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) > str(service_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

In [122]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

,id,id_samochodu,id_klienta,id_pracownika,cena,data_zaksiegowania
0,1,1,1,2,44220.0,2014-01-02
1,2,2,2,2,50010.0,2014-01-03
2,3,3,3,3,34130.0,2014-01-08
3,4,4,4,3,65810.0,2014-01-08
4,5,5,5,3,29100.0,2014-01-09
...,...,...,...,...,...,...
928,929,982,1507,6,30980.0,2020-12-20
929,930,983,432,3,19370.0,2020-12-24
930,931,984,1510,3,26510.0,2020-12-26
931,932,985,1511,3,19980.0,2020-12-29


# Sprzedaż samochodów

In [124]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksiegowania'] = transaction_dates_sell

In [128]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania
0,1,10,21,2,2014-02-01
1,2,14,22,2,2014-02-04
2,3,18,23,3,2014-02-05
3,4,13,24,2,2014-02-05
4,5,2,25,3,2014-02-06
...,...,...,...,...,...
922,923,973,171,2,2020-12-15
923,924,971,1508,2,2020-12-22
924,925,982,605,6,2020-12-22
925,926,977,873,2,2020-12-23


In [137]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    print(b_price)
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

df_sell['cena'] = sell_prices

35090.0
27310.0
82030.0
20950.0
50010.0
59380.0
29100.0
44220.0
34130.0
62890.0
15940.0
120340.0
35710.0
35740.0
50800.0
60380.0
65810.0
25120.0
30570.0
37590.0
4740.0
85180.0
40580.0
34980.0
61130.0
26720.0
26250.0
30100.0
19480.0
47540.0
35530.0
37610.0
19980.0
68960.0
74170.0
13470.0
30670.0
55460.0
42780.0
30410.0
106480.0
36200.0
20930.0
nan
61600.0
22160.0
40790.0
32690.0
38040.0
363850.0
17070.0
30020.0
39850.0
55690.0
38950.0
39050.0
inf
143850.0
nan
26000.0
61280.0
41670.0
46030.0
48380.0
nan
38100.0
46610.0
44650.0
38580.0
52900.0
19780.0
62350.0
65450.0
64890.0
35130.0
44520.0
nan
65490.0
28080.0
nan
16350.0
17390.0
179450.0
41540.0
31280.0
nan
nan
40330.0
76080.0
126130.0
inf
nan
120430.0
65040.0
39200.0
15680.0
126120.0
85020.0
61730.0
22070.0
51520.0
66630.0
nan
51190.0
100440.0
30290.0
42160.0
51450.0
59660.0
43540.0
nan
45220.0
29980.0
53560.0
30290.0
35920.0
45470.0
17140.0
44960.0
64180.0
64630.0
34190.0
33750.0
23090.0
43330.0
95170.0
40790.0
47260.0
37270.0
112990.0

In [138]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,10,21,2,2014-02-01,46400.0
1,2,14,22,2,2014-02-04,37900.0
2,3,18,23,3,2014-02-05,109700.0
3,4,13,24,2,2014-02-05,27100.0
4,5,2,25,3,2014-02-06,70100.0
...,...,...,...,...,...,...
922,923,973,171,2,2020-12-15,60700.0
923,924,971,1508,2,2020-12-22,226200.0
924,925,982,605,6,2020-12-22,44900.0
925,926,977,873,2,2020-12-23,82500.0


# Użyte części

In [ ]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(32), array([17, 17])), (np.int64(39), array([29])), (np.int64(72), array([8])), (np.int64(73), array([52])), (np.int64(77), array([56, 26, 32])), (np.int64(83), array([43])), (np.int64(88), array([52, 18, 25])), (np.int64(89), array([63])), (np.int64(98), array([53])), (np.int64(101), array([47])), (np.int64(103), array([49, 27])), (np.int64(113), array([7])), (np.int64(116), array([ 8, 44])), (np.int64(137), array([41, 21, 36])), (np.int64(139), array([18, 29])), (np.int64(149), array([0])), (np.int64(152), array([32])), (np.int64(183), array([36])), (np.int64(191), array([21, 32,  5])), (np.int64(192), array([29, 31])), (np.int64(195), array([ 9, 46])), (np.int64(201), array([53])), (np.int64(211), array([57, 17])), (np.int64(216), array([37])), (np.int64(217), array([62, 49])), (np.int64(224), array([60, 35])), (np.int64(238), array([16])), (np.int64(246), array([24])), (np.int64(251), array([41])), (np.int64(257), array([27, 46])), (np.int64(263), array([51])), (np.int64

In [ ]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id+1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [ ]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Alternator,Opel Karl,152,1,1
1,Alternator,Opel Karl,146,1,1
2,Deska rozdzielcza,Opel Crossland X,83,2,1
3,Wewnętrzna klamka drzwi,Volkswagen Polo,6,3,1
4,Zbiornik paliwa,Opel Corsa,273,4,1
...,...,...,...,...,...
157,Wał korbowy,Skoda Fabia,668,106,1
158,Dystrybutor,Skoda Fabia,236,106,1
159,Drzwi prawe tylne,Citroen C1 szary,328,107,1
160,Miska olejowa,Fiat Punto Evo,324,108,1


# Wyposażenie

In [ ]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [ ]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Alternator,Opel Karl,152.0
2,2,Deska rozdzielcza,Opel Crossland X,83.0
3,3,Wewnętrzna klamka drzwi,Volkswagen Polo,6.0
4,4,Zbiornik paliwa,Opel Corsa,273.0
5,5,Kierownica,smart forFour,62.0
...,...,...,...,...
157,125,Wał korbowy,Skoda Fabia,668.0
158,126,Dystrybutor,Skoda Fabia,236.0
159,127,Drzwi prawe tylne,Citroen C1 szary,328.0
160,128,Miska olejowa,Fiat Punto Evo,324.0


# Usługi - uzupełnienie

In [ ]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [ ]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,32,62,8,2014-03-22,390
1,2,39,75,9,2014-04-03,110
2,3,72,137,4,2014-06-20,10
3,4,73,139,9,2014-06-21,360
4,5,77,147,7,2014-06-29,280
...,...,...,...,...,...,...
104,105,175,323,9,2020-07-14,80
105,106,931,137,7,2020-09-06,1730
106,107,938,1412,8,2020-09-25,430
107,108,958,876,8,2020-11-18,430


# Użyte części - uzupełnienie

In [ ]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [ ]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,1,1
2,3,2,2,1
3,4,3,3,1
4,5,4,4,1
...,...,...,...,...
157,158,106,125,1
158,159,106,126,1
159,160,107,127,1
160,161,108,128,1


# Zakup części

In [ ]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksiegowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'data_zakupu', 'ilość', 'data_zaksięgowania'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))
    service_date = get_service_date(part_id)

    ##### DO DODANIA
    # generowanie daty zakupu części na podstawie purchase_date
    # generowanie daty zaksięgowania transakcji na podstawie purchase_date
    #####
    
    # row = [i+1, part_id, total_cost, purchase_date, quantity, posting_date]
    # parts_purchase_pd.loc[i] = row


In [ ]:
# parts_purchase_pd